In [16]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

## 1. ETL:

In [5]:
#extract data via Dune API
def extract_from_dune(api_key, query_id):
    dune = DuneClient(api_key)
    query = QueryBase(query_id = query_id)
    query_result = dune.run_query_dataframe(query=query)
    return query_result

In [4]:
#get data from csv file
def extract_from_csv(file_path):
    df = pd.read_csv(file_path)
    return df

In [3]:
#load data to csv file
def load(file_path, data_to_load):
    data_to_load.to_csv(file_path)

In [18]:
#pre-process the data
def transform(myDf):
    #drop unwanted column
    if 'Unnamed: 0' in myDf.columns:
        myDf = myDf.drop(columns=['Unnamed: 0'])
    #convert day to datetime type
    myDf['day'] = pd.to_datetime(myDf['day'])
    myDf = myDf.sort_values(['token', 'day'])
    #sort data by day
    myDf['day'] = myDf['day'].sort_values()
    #handle missing data
    myDf = myDf.fillna(0)
    #filter out "scam" or "malicious" coins
    return myDf

In [11]:
#create monthly stats dataframe
def aggregate_to_monthly(df):

    df['day'] = pd.to_datetime(df['day'])
    
    df['month'] = df['day'].dt.to_period('M')
    
    # Aggregate by token, contract_address, and month
    monthly_df = df.groupby(['token', 'contract_address', 'month']).agg({
        'trading_volume': 'mean',
        'trading_pairs': 'mean',           
        'total_traders': 'mean',         
        'avg_trading_size': 'mean',      
        'daily_price': 'mean'              
    }).reset_index()
    
    # Rename columns for readability
    monthly_df = monthly_df.rename(columns={
        'trading_volume': 'avg_monthly_trading_volume',
        'trading_pairs': 'avg_trading_pairs',
        'total_traders': 'avg_total_traders',
        'avg_trading_size': 'avg_monthly_trading_size',
        'daily_price': 'avg_monthly_price'
    })
    
    return monthly_df

In [13]:
#define unqualified token
def unqualified_token(df, vol_threshold, trader_threshold, trading_pair_threshold):
    un_df = df[(df['avg_monthly_trading_volume'] < vol_threshold) | 
               (df['avg_total_traders'] < trader_threshold) | 
               (df['avg_trading_pairs'] < trading_pair_threshold)]
    unqualified_list = un_df['token'].unique().tolist()
    return unqualified_list

In [17]:
#create correlation heatmap between numeric features
def pairwise_corr(df):
    numeric_df = df.select_dtypes(include=['number'])
    corr = numeric_df.corr(method='pearson')
    plt.figure(figsize=(12,9))
    sns.heatmap(corr, annot=True, cmap='coolwarm')
    plt.title('Features heatmap correlation')
    plt.show()

In [ ]:
#create pairwise plots:
def pariwise_plot(df):
    numeric_df = df.select_dtypes(include=['number'])
    